In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


In [2]:

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
df_green=spark.read.parquet("C:/data_engineering_project/06_Batch_processing_with_spark/data/pq/green/*/*")
df_green.show()

In [4]:
df_yellow=spark.read.parquet("C:/data_engineering_project/06_Batch_processing_with_spark/data/pq/yellow/*/*")
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-03-23 13:15:29|  2019-03-23 13:19:57|              1|          1.0|         1|                 N|         237|         141|           1|        5.5|  2.5|    0.5|       1.5|         0.0|                  0.3

In [5]:
# renaming some columns green data
df_green=df_green \
           .withColumnRenamed('lpep_pickup_datetime','pickup_datetime') \
           .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')

In [6]:
# renaming some columns in yellow dataframe
df_yellow=df_yellow \
           .withColumnRenamed('tpep_pickup_datetime','pickup_datetime') \
           .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

In [7]:
df_green.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [8]:
# lis with common column from the 2 dataframes and in the same order as original dataframes.
common_columns=[]

yellow_columns=set(df_yellow.columns)
for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [9]:
df_green_sel=df_green \
             .select(common_columns) \
             .withColumn('service_type',F.lit('green'))

In [10]:
df_yellow_sel=df_yellow \
             .select(common_columns) \
             .withColumn('service_type',F.lit('yellow'))

In [11]:
# merging dataframes to one
df_trips=df_green_sel.union(df_yellow_sel)

In [12]:
df_trips.groupBy('service_type').count().show()

+------------+---------+
|service_type|    count|
+------------+---------+
|       green|  7778101|
|      yellow|109047518|
+------------+---------+



In [13]:
# using sql queries
# convert dataframe to a table
df_trips.createOrReplaceTempView('Trips_data')

In [14]:
df_trips.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2|2019-01-29 15:36:35|2019-01-29 15:45:15|                 N|         1|          40|         181|              1|         1.12|        7.5|  0.0|    0.5|      1.66|         0.0|       

In [15]:
spark.sql ("""
SELECT service_type,count(service_type) AS Count 
FROM Trips_data
GROUP BY service_type
""").show()

+------------+---------+
|service_type|    Count|
+------------+---------+
|       green|  7778101|
|      yellow|109047518|
+------------+---------+



In [16]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    Trips_data
GROUP BY
    1, 2, 3
""")

In [17]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|         200|2019-

In [19]:
df_result.coalesce(1).write.mode('overwrite').parquet('C:/data_engineering_project/06_Batch_processing_with_spark/data/report/revenue')
